In [1]:
import pandas as pd 
import numpy as np

In [2]:
df_raw = pd.read_csv('C:\\Users\\USER\\Desktop\\FYP\\fyp-sandbox-2\\Data\\RAW_DATA\\DRUGS/DRUG24Q4.txt', sep='$')
df_raw

C:\Users\USER\AppData\Local\Temp\ipykernel_4744\1720144281.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv('C:\\Users\\USER\\Desktop\\FYP\\fyp-sandbox-2\\Data\\RAW_DATA\\DRUGS/DRUG24Q4.txt', sep='$')


,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_num,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq
0,100100247,10010024,1,PS,ADALIMUMAB,ADALIMUMAB,1,Subcutaneous,NaN,NaN,NaN,Y,NaN,Not Available,NaN,125057.0,40.0,MG,NaN,QOW
1,100100247,10010024,2,SS,ADALIMUMAB,ADALIMUMAB,1,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,125057.0,NaN,NaN,NaN,NaN
2,100100247,10010024,3,SS,CYCLOSPORINE,CYCLOSPORINE,1,Unknown,NaN,NaN,NaN,D,NaN,Not Available,NaN,65003.0,NaN,NaN,NaN,NaN
3,100100247,10010024,4,SS,REMICADE,INFLIXIMAB,1,Intravenous (not otherwise specified),6 infusions at 8 weeks interval,NaN,NaN,Y,NaN,Not Available,NaN,NaN,NaN,NaN,NaN,NaN
4,100100247,10010024,5,SS,ETANERCEPT,ETANERCEPT,1,Subcutaneous,NaN,NaN,NaN,Y,NaN,Not Available,NaN,NaN,100.0,MG,NaN,/WK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030933,99696784,9969678,12,SS,DAPTOMYCIN,DAPTOMYCIN,1,Unknown,"500 mg, Daily",NaN,NaN,Y,NaN,NaN,NaN,NaN,500.0,MG,NaN,NaN
2030934,99696784,9969678,13,SS,PANTOPRAZOLE SODIUM I.V.,PANTOPRAZOLE SODIUM,1,Unknown,"40 mg, UNK",NaN,NaN,U,U,unk,NaN,NaN,40.0,MG,Injection,NaN
2030935,99696784,9969678,14,C,ACETAMINOPHEN\CODEINE PHOSPHATE,ACETAMINOPHEN\CODEINE PHOSPHATE,1,Unknown,"500/30 mg, daily",NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2030936,99696784,9969678,15,C,METHYLPREDNISOLONE,METHYLPREDNISOLONE,1,Unknown,"80 mg, bid",NaN,NaN,Y,NaN,NaN,NaN,NaN,80.0,MG,NaN,Q12H


### Creating a logic for replacing the missing prod_ai values

In [3]:
df_raw.isna().sum()

primaryid              0
caseid                 0
drug_seq               0
role_cod               0
drugname               7
prod_ai            30030
val_vbm                0
route             641361
dose_vbm          619190
cum_dose_chr     1993811
cum_dose_unit    1993811
dechal            822598
rechal           1853194
lot_num          1530761
exp_dt           2029280
nda_num          1378374
dose_amt         1192311
dose_unit        1192311
dose_form         994497
dose_freq        1606951
dtype: int64

In [5]:
df_raw.columns

Index(['primaryid', 'caseid', 'drug_seq', 'role_cod', 'drugname', 'prod_ai',
       'val_vbm', 'route', 'dose_vbm', 'cum_dose_chr', 'cum_dose_unit',
       'dechal', 'rechal', 'lot_num', 'exp_dt', 'nda_num', 'dose_amt',
       'dose_unit', 'dose_form', 'dose_freq'],
      dtype='object')

In [4]:
#First let's remove the unwanted cols
df_raw_2 = df_raw[['primaryid', 'caseid', 'drug_seq', 'role_cod', 'drugname', 'prod_ai']]
df_raw_2.isna().sum()

primaryid        0
caseid           0
drug_seq         0
role_cod         0
drugname         7
prod_ai      30030
dtype: int64

In [7]:
df_raw_2[df_raw_2['drugname'].isna() == True]

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai
945010,244269481,24426948,2,C,NaN,UNSPECIFIED INGREDIENT
1991280,247842001,24784200,2,C,NaN,UNSPECIFIED INGREDIENT
1991281,247842001,24784200,3,C,NaN,UNSPECIFIED INGREDIENT
1991282,247842001,24784200,4,C,NaN,UNSPECIFIED INGREDIENT
1991284,247842011,24784201,2,C,NaN,UNSPECIFIED INGREDIENT
1991285,247842011,24784201,3,C,NaN,UNSPECIFIED INGREDIENT
1991286,247842011,24784201,4,C,NaN,UNSPECIFIED INGREDIENT


If the drugname is missing, it can be taken out (dropped). Since the drug is unspecified, it would not affect the analysis or anything by a one particular case having missed drugs

In [8]:
#Step 01: remove any records without the drugname
cods = df_raw_2[df_raw_2['drugname'].isna() == True]['primaryid']
temp_1= df_raw[df_raw['primaryid'].isin(cods)]
temp_1[temp_1['drugname'].isna()==True]

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_num,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq
945010,244269481,24426948,2,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991280,247842001,24784200,2,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991281,247842001,24784200,3,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991282,247842001,24784200,4,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991284,247842011,24784201,2,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991285,247842011,24784201,3,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991286,247842011,24784201,4,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


So it's confirmed that the records with no drugname have no other info. And it's safe to remove all of them

In [5]:
#remove all the drugnames which have missing values
df_raw_2 = df_raw_2[df_raw_2['drugname'].isna() == False]
df_raw_2.isna().sum()

primaryid        0
caseid           0
drug_seq         0
role_cod         0
drugname         0
prod_ai      30030
dtype: int64

### Replacing Prod_AI

In [14]:
#Making a unique values of drugnames and prod_ai
lookup_drugs = df_raw_2[['drugname', 'prod_ai']]
lookup_drugs

,drugname,prod_ai
0,ADALIMUMAB,ADALIMUMAB
1,ADALIMUMAB,ADALIMUMAB
2,CYCLOSPORINE,CYCLOSPORINE
3,REMICADE,INFLIXIMAB
4,ETANERCEPT,ETANERCEPT
...,...,...
2030933,DAPTOMYCIN,DAPTOMYCIN
2030934,PANTOPRAZOLE SODIUM I.V.,PANTOPRAZOLE SODIUM
2030935,ACETAMINOPHEN\CODEINE PHOSPHATE,ACETAMINOPHEN\CODEINE PHOSPHATE
2030936,METHYLPREDNISOLONE,METHYLPREDNISOLONE


In [15]:
lookup_drugs['drugname_l'] = lookup_drugs['drugname'].str.lower()
lookup_drugs['prod_ai_l'] = lookup_drugs['prod_ai'].str.lower()
lookup_drugs

C:\Users\USER\AppData\Local\Temp\ipykernel_4744\3810363072.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lookup_drugs['drugname_l'] = lookup_drugs['drugname'].str.lower()
C:\Users\USER\AppData\Local\Temp\ipykernel_4744\3810363072.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lookup_drugs['prod_ai_l'] = lookup_drugs['prod_ai'].str.lower()


,drugname,prod_ai,drugname_l,prod_ai_l
0,ADALIMUMAB,ADALIMUMAB,adalimumab,adalimumab
1,ADALIMUMAB,ADALIMUMAB,adalimumab,adalimumab
2,CYCLOSPORINE,CYCLOSPORINE,cyclosporine,cyclosporine
3,REMICADE,INFLIXIMAB,remicade,infliximab
4,ETANERCEPT,ETANERCEPT,etanercept,etanercept
...,...,...,...,...
2030933,DAPTOMYCIN,DAPTOMYCIN,daptomycin,daptomycin
2030934,PANTOPRAZOLE SODIUM I.V.,PANTOPRAZOLE SODIUM,pantoprazole sodium i.v.,pantoprazole sodium
2030935,ACETAMINOPHEN\CODEINE PHOSPHATE,ACETAMINOPHEN\CODEINE PHOSPHATE,acetaminophen\codeine phosphate,acetaminophen\codeine phosphate
2030936,METHYLPREDNISOLONE,METHYLPREDNISOLONE,methylprednisolone,methylprednisolone


In [16]:
lookup_drugs.drop(['drugname', 'prod_ai'], axis=1, inplace=True)
lookup_drugs.columns

C:\Users\USER\AppData\Local\Temp\ipykernel_4744\2790644918.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lookup_drugs.drop(['drugname', 'prod_ai'], axis=1, inplace=True)


Index(['drugname_l', 'prod_ai_l'], dtype='object')

In [17]:
lookup_drugs.drop_duplicates(inplace=True)
lookup_drugs

C:\Users\USER\AppData\Local\Temp\ipykernel_4744\1158587578.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lookup_drugs.drop_duplicates(inplace=True)


,drugname_l,prod_ai_l
0,adalimumab,adalimumab
2,cyclosporine,cyclosporine
3,remicade,infliximab
4,etanercept,etanercept
5,methotrexate,methotrexate
...,...,...
2030682,apo diltiaz cd,NaN
2030701,acetyldihydrocodeine hydrochloride,acetyldihydrocodeine hydrochloride
2030777,viraferonpeg,peginterferon alfa-2b
2030859,myoflexin,NaN


In [19]:
lookup_drugs[lookup_drugs['prod_ai_l'].isna()==True]

,drugname_l,prod_ai_l
304,tylenol nighttime,NaN
567,inibace plus,NaN
1441,reactine,NaN
2588,camomile,NaN
2627,pqq,NaN
...,...,...
2028775,methylinphdate,NaN
2029039,tenofovir sandoz [tenofovir disoproxil],NaN
2030133,amikacine ^ion^,NaN
2030682,apo diltiaz cd,NaN


In [27]:
#Let's fill the missing values in the lookup table first
def fill_missing(df):
    if pd.isna(df['prod_ai_l']):
        return 'NA'
    else:
        return df['prod_ai_l']


In [28]:
lookup_drugs['corr_prod_ai'] = lookup_drugs.apply(fill_missing, axis=1)
lookup_drugs

C:\Users\USER\AppData\Local\Temp\ipykernel_4744\2244251935.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lookup_drugs['corr_prod_ai'] = lookup_drugs.apply(fill_missing, axis=1)


,drugname_l,prod_ai_l,corr_prod_ai
0,adalimumab,adalimumab,adalimumab
2,cyclosporine,cyclosporine,cyclosporine
3,remicade,infliximab,infliximab
4,etanercept,etanercept,etanercept
5,methotrexate,methotrexate,methotrexate
...,...,...,...
2030682,apo diltiaz cd,NaN,NA
2030701,acetyldihydrocodeine hydrochloride,acetyldihydrocodeine hydrochloride,acetyldihydrocodeine hydrochloride
2030777,viraferonpeg,peginterferon alfa-2b,peginterferon alfa-2b
2030859,myoflexin,NaN,NA


In [29]:
lookup_drugs.isna().sum()

drugname_l          0
prod_ai_l       16022
corr_prod_ai        0
dtype: int64

In [63]:
lookup_drugs[lookup_drugs['corr_prod_ai'] == 'NA']

,drugname_l,prod_ai_l,corr_prod_ai
304,tylenol nighttime,NaN,NA
567,inibace plus,NaN,NA
1441,reactine,NaN,NA
2588,camomile,NaN,NA
2627,pqq,NaN,NA
...,...,...,...
2028775,methylinphdate,NaN,NA
2029039,tenofovir sandoz [tenofovir disoproxil],NaN,NA
2030133,amikacine ^ion^,NaN,NA
2030682,apo diltiaz cd,NaN,NA


In [64]:
#Now let's make a function to substitue missing values
df_raw_2['drugname_1'] = df_raw_2['drugname'].str.lower()
df_raw_2['prod_ai_1'] = df_raw_2['prod_ai'].str.lower()
df_raw_2.head()

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,drugname_l,drugname_1,prod_ai_1
0,100100247,10010024,1,PS,ADALIMUMAB,ADALIMUMAB,adalimumab,adalimumab,adalimumab
1,100100247,10010024,2,SS,ADALIMUMAB,ADALIMUMAB,adalimumab,adalimumab,adalimumab
2,100100247,10010024,3,SS,CYCLOSPORINE,CYCLOSPORINE,cyclosporine,cyclosporine,cyclosporine
3,100100247,10010024,4,SS,REMICADE,INFLIXIMAB,remicade,remicade,infliximab
4,100100247,10010024,5,SS,ETANERCEPT,ETANERCEPT,etanercept,etanercept,etanercept


In [67]:
df_raw_5 = pd.merge(left = df_raw_2, right = lookup_drugs, how = 'left', left_on= 'drugname_1', right_on= 'drugname_l')
df_raw_5

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,drugname_l_x,drugname_1,prod_ai_1,drugname_l_y,prod_ai_l,corr_prod_ai
0,100100247,10010024,1,PS,ADALIMUMAB,ADALIMUMAB,adalimumab,adalimumab,adalimumab,adalimumab,adalimumab,adalimumab
1,100100247,10010024,2,SS,ADALIMUMAB,ADALIMUMAB,adalimumab,adalimumab,adalimumab,adalimumab,adalimumab,adalimumab
2,100100247,10010024,3,SS,CYCLOSPORINE,CYCLOSPORINE,cyclosporine,cyclosporine,cyclosporine,cyclosporine,cyclosporine,cyclosporine
3,100100247,10010024,3,SS,CYCLOSPORINE,CYCLOSPORINE,cyclosporine,cyclosporine,cyclosporine,cyclosporine,NaN,NA
4,100100247,10010024,4,SS,REMICADE,INFLIXIMAB,remicade,remicade,infliximab,remicade,infliximab,infliximab
...,...,...,...,...,...,...,...,...,...,...,...,...
2643841,99696784,9969678,12,SS,DAPTOMYCIN,DAPTOMYCIN,daptomycin,daptomycin,daptomycin,daptomycin,daptomycin,daptomycin
2643842,99696784,9969678,13,SS,PANTOPRAZOLE SODIUM I.V.,PANTOPRAZOLE SODIUM,pantoprazole sodium i.v.,pantoprazole sodium i.v.,pantoprazole sodium,pantoprazole sodium i.v.,pantoprazole sodium,pantoprazole sodium
2643843,99696784,9969678,14,C,ACETAMINOPHEN\CODEINE PHOSPHATE,ACETAMINOPHEN\CODEINE PHOSPHATE,acetaminophen\codeine phosphate,acetaminophen\codeine phosphate,acetaminophen\codeine phosphate,acetaminophen\codeine phosphate,acetaminophen\codeine phosphate,acetaminophen\codeine phosphate
2643844,99696784,9969678,15,C,METHYLPREDNISOLONE,METHYLPREDNISOLONE,methylprednisolone,methylprednisolone,methylprednisolone,methylprednisolone,methylprednisolone,methylprednisolone


In [68]:
df_raw_5.isna().sum()

primaryid            0
caseid               0
drug_seq             0
role_cod             0
drugname             0
prod_ai          30922
drugname_l_x         0
drugname_1           0
prod_ai_1        30922
drugname_l_y         0
prod_ai_l       628717
corr_prod_ai         0
dtype: int64

In [69]:
df_raw_5[df_raw_5['corr_prod_ai'] == 'NA']

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,drugname_l_x,drugname_1,prod_ai_1,drugname_l_y,prod_ai_l,corr_prod_ai
3,100100247,10010024,3,SS,CYCLOSPORINE,CYCLOSPORINE,cyclosporine,cyclosporine,cyclosporine,cyclosporine,NaN,NA
7,100100247,10010024,6,SS,METHOTREXATE,METHOTREXATE,methotrexate,methotrexate,methotrexate,methotrexate,NaN,NA
12,100100247,10010024,10,C,CLOBETASOL,CLOBETASOL,clobetasol,clobetasol,clobetasol,clobetasol,NaN,NA
15,100373859,10037385,2,I,OMEPRAZOLE,OMEPRAZOLE,omeprazole,omeprazole,omeprazole,omeprazole,NaN,NA
18,100373859,10037385,4,C,ZINC,ZINC,zinc,zinc,zinc,zinc,NaN,NA
...,...,...,...,...,...,...,...,...,...,...,...,...
2643827,99696784,9969678,2,C,LEVOFLOXACIN,LEVOFLOXACIN,levofloxacin,levofloxacin,levofloxacin,levofloxacin,NaN,NA
2643831,99696784,9969678,5,PS,NAPROXEN,NAPROXEN,naproxen,naproxen,naproxen,naproxen,NaN,NA
2643833,99696784,9969678,6,SS,PANTOPRAZOLE,PANTOPRAZOLE,pantoprazole,pantoprazole,pantoprazole,pantoprazole,NaN,NA
2643835,99696784,9969678,7,SS,ACETAMINOPHEN,ACETAMINOPHEN,acetaminophen,acetaminophen,acetaminophen,acetaminophen,NaN,NA


In [32]:
df_raw_2.drop('prod_ai_l', axis=1, inplace=True)
        


In [35]:
#Merging
df_raw_3 = pd.merge(left = df_raw_2, right = lookup_drugs, how = 'left', left_on= 'drugname_l', right_on= 'drugname_l')
df_raw_3

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,drugname_l,prod_ai_l,corr_prod_ai
0,100100247,10010024,1,PS,ADALIMUMAB,ADALIMUMAB,adalimumab,adalimumab,adalimumab
1,100100247,10010024,2,SS,ADALIMUMAB,ADALIMUMAB,adalimumab,adalimumab,adalimumab
2,100100247,10010024,3,SS,CYCLOSPORINE,CYCLOSPORINE,cyclosporine,cyclosporine,cyclosporine
3,100100247,10010024,3,SS,CYCLOSPORINE,CYCLOSPORINE,cyclosporine,NaN,NA
4,100100247,10010024,4,SS,REMICADE,INFLIXIMAB,remicade,infliximab,infliximab
...,...,...,...,...,...,...,...,...,...
2643841,99696784,9969678,12,SS,DAPTOMYCIN,DAPTOMYCIN,daptomycin,daptomycin,daptomycin
2643842,99696784,9969678,13,SS,PANTOPRAZOLE SODIUM I.V.,PANTOPRAZOLE SODIUM,pantoprazole sodium i.v.,pantoprazole sodium,pantoprazole sodium
2643843,99696784,9969678,14,C,ACETAMINOPHEN\CODEINE PHOSPHATE,ACETAMINOPHEN\CODEINE PHOSPHATE,acetaminophen\codeine phosphate,acetaminophen\codeine phosphate,acetaminophen\codeine phosphate
2643844,99696784,9969678,15,C,METHYLPREDNISOLONE,METHYLPREDNISOLONE,methylprednisolone,methylprednisolone,methylprednisolone


In [36]:
df_raw_3 = df_raw_3.drop(['drugname', 'prod_ai', 'prod_ai_l'], axis=1)
df_raw_3

,primaryid,caseid,drug_seq,role_cod,drugname_l,corr_prod_ai
0,100100247,10010024,1,PS,adalimumab,adalimumab
1,100100247,10010024,2,SS,adalimumab,adalimumab
2,100100247,10010024,3,SS,cyclosporine,cyclosporine
3,100100247,10010024,3,SS,cyclosporine,NA
4,100100247,10010024,4,SS,remicade,infliximab
...,...,...,...,...,...,...
2643841,99696784,9969678,12,SS,daptomycin,daptomycin
2643842,99696784,9969678,13,SS,pantoprazole sodium i.v.,pantoprazole sodium
2643843,99696784,9969678,14,C,acetaminophen\codeine phosphate,acetaminophen\codeine phosphate
2643844,99696784,9969678,15,C,methylprednisolone,methylprednisolone


In [37]:
df_raw_3.isna().sum()

primaryid       0
caseid          0
drug_seq        0
role_cod        0
drugname_l      0
corr_prod_ai    0
dtype: int64

In [39]:
#Checking how many records are not having NA values
len(df_raw_3[df_raw_3['corr_prod_ai'] == 'NA'])

628717

In [40]:
df_raw_2[df_raw_2['prod_ai'].isna() == True]

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,drugname_l
304,1046045537,10460455,59,C,TYLENOL NIGHTTIME,NaN,tylenol nighttime
567,1059192435,10591924,41,C,INIBACE PLUS,NaN,inibace plus
1441,1107812716,11078127,30,C,Reactine,NaN,reactine
2588,114733203,11473320,16,C,CAMOMILE,NaN,camomile
2627,114798566,11479856,35,C,PQQ,NaN,pqq
...,...,...,...,...,...,...,...
2029309,774113178,7741131,22,C,DIAMICRON,NaN,diamicron
2030133,91440942,9144094,30,C,AMIKACINE ^ION^,NaN,amikacine ^ion^
2030682,977806967,9778069,66,C,APO DILTIAZ CD,NaN,apo diltiaz cd
2030796,98318603,9831860,26,C,Abacavir + lamivudina,NaN,abacavir + lamivudina


In [41]:
df_raw_3[df_raw_3['corr_prod_ai'] == 'NA']

,primaryid,caseid,drug_seq,role_cod,drugname_l,corr_prod_ai
3,100100247,10010024,3,SS,cyclosporine,NA
7,100100247,10010024,6,SS,methotrexate,NA
12,100100247,10010024,10,C,clobetasol,NA
15,100373859,10037385,2,I,omeprazole,NA
18,100373859,10037385,4,C,zinc,NA
...,...,...,...,...,...,...
2643827,99696784,9969678,2,C,levofloxacin,NA
2643831,99696784,9969678,5,PS,naproxen,NA
2643833,99696784,9969678,6,SS,pantoprazole,NA
2643835,99696784,9969678,7,SS,acetaminophen,NA


In [43]:
lookup_2 = df_raw_2[['drugname', 'prod_ai']]
lookup_2.drop_duplicates(inplace=True)
lookup_2

C:\Users\USER\AppData\Local\Temp\ipykernel_4744\1610249892.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lookup_2.drop_duplicates(inplace=True)


,drugname,prod_ai
0,ADALIMUMAB,ADALIMUMAB
2,CYCLOSPORINE,CYCLOSPORINE
3,REMICADE,INFLIXIMAB
4,ETANERCEPT,ETANERCEPT
5,METHOTREXATE,METHOTREXATE
...,...,...
2030682,APO DILTIAZ CD,NaN
2030701,ACETYLDIHYDROCODEINE HYDROCHLORIDE,ACETYLDIHYDROCODEINE HYDROCHLORIDE
2030777,VIRAFERONPEG,PEGINTERFERON ALFA-2B
2030859,MYOFLEXIN,NaN


In [45]:
lookup_2.rename(columns={'drugname':'drugname_l', 'prod_ai':'prod_ai_l'}, inplace=True)
lookup_2.head()

C:\Users\USER\AppData\Local\Temp\ipykernel_4744\1136809954.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lookup_2.rename(columns={'drugname':'drugname_l', 'prod_ai':'prod_ai_l'}, inplace=True)


,drugname_l,prod_ai_l
0,ADALIMUMAB,ADALIMUMAB
2,CYCLOSPORINE,CYCLOSPORINE
3,REMICADE,INFLIXIMAB
4,ETANERCEPT,ETANERCEPT
5,METHOTREXATE,METHOTREXATE


In [ ]:
#These are the drugs with more than 1 count of prod_ai values
temp_1 = lookup_2.groupby('drugname_l').size().reset_index(name='Count').sort_values(by=['Count'], ascending= False)
temp_1 = temp_1[temp_1['Count'] > 1]
temp_1

,drugname_l,Count
7753,DEXTROSE,3
3672,BISOPROLOL,3
15945,MONTELUKAST,3
15949,MONTELUKAST SODIUM,3
19498,PRAZOSIN,3
...,...,...
25889,ZINC SULFATE,2
25955,ZOLOFT,2
25957,ZOLPIDEM,2
26039,ZYPREXA,2


In [62]:
#Extracting the drug names with more than 1 count
drug_counts = list(temp_1['drugname_l'])
drug_counts

['DEXTROSE',
 'BISOPROLOL',
 'MONTELUKAST',
 'MONTELUKAST SODIUM',
 'PRAZOSIN',
 'SUMATRIPTAN',
 'DEPAKOTE',
 'ADEMPAS',
 'FLUOXETINE',
 'ESOMEPRAZOLE',
 'GAS-X',
 'COLLAGEN',
 'FOSAPREPITANT',
 'CALCIUM',
 'AZILECT',
 'FOSAPREPITANT DIMEGLUMINE',
 'CLOBETASOL',
 'CYANOCOBALAMIN',
 'CORLANOR',
 'JEMPERLI',
 'JUVEDERM',
 'CYCLOSPORINE',
 'DALIRESP',
 'CIPROFLOXACIN',
 'KLONOPIN',
 'AUBAGIO',
 'CARBOPLATIN',
 'DIALYVITE',
 'FIORICET',
 'BIJUVA',
 'DILAUDID',
 'FOLIC ACID',
 'DEXAMETHASONE',
 'CYMBALTA',
 'EZETIMIBE',
 'DOCUSATE',
 'EXJADE',
 'BACLOFEN',
 'BUSPAR',
 'BUSPIRONE',
 'CALCIUM CITRATE',
 'CHROMIUM',
 'CEFUROXIME',
 'DEXTROAMPHETAMINE SACCHARATE, AMPHETAMINE ASPARTATE, DEXTROAMPHETAMINE SULFATE AND AMPHETAMINE SULFATE',
 'FLUTICASONE',
 'ALPHA LIPOIC ACID',
 'CHROMIUM PICOLINATE',
 'KRILL OIL',
 'ICLUSIG',
 'INSULIN GLARGINE',
 'JARDIANCE',
 'ATENOLOL',
 'DILTIAZEM',
 'IRON',
 'ARMOUR THYROID',
 'BACTRIM',
 'BACTRIM DS',
 'DIFLUCAN',
 'ALPRAZOLAM',
 'ADVAIR HFA',
 'ELIQUIS',
 '

In [ ]:
#Replacing
def replace_prodAI(df):
    if (not pd.isna(df['drugname'])) and (df['drugname'].isin(drug_counts)) and (pd.isna(df['prod_ai'])):
        return ''

In [60]:
df_raw_2[df_raw_2['drugname'] == 'MONTELUKAST']['prod_ai'].unique()

array(['MONTELUKAST', 'MONTELUKAST SODIUM', nan], dtype=object)

In [50]:
def lookup_trans(df):
    if (not pd.isna(df['drugname_l'])) and (pd.isna(df['prod_ai_l'])):
        return 'remove'
    else:
        return 'Keep'

In [51]:
lookup_2['class'] = lookup_2.apply(lookup_trans, axis=1)
lookup_2

C:\Users\USER\AppData\Local\Temp\ipykernel_4744\3336082698.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lookup_2['class'] = lookup_2.apply(lookup_trans, axis=1)


,drugname_l,prod_ai_l,class
0,ADALIMUMAB,ADALIMUMAB,Keep
2,CYCLOSPORINE,CYCLOSPORINE,Keep
3,REMICADE,INFLIXIMAB,Keep
4,ETANERCEPT,ETANERCEPT,Keep
5,METHOTREXATE,METHOTREXATE,Keep
...,...,...,...
2030682,APO DILTIAZ CD,NaN,remove
2030701,ACETYLDIHYDROCODEINE HYDROCHLORIDE,ACETYLDIHYDROCODEINE HYDROCHLORIDE,Keep
2030777,VIRAFERONPEG,PEGINTERFERON ALFA-2B,Keep
2030859,MYOFLEXIN,NaN,remove


In [46]:
df_raw_4 = pd.merge(left=df_raw_2, right= lookup_2, how='left', left_on='drugname', right_on='drugname_l')
df_raw_4.head()

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,drugname_l_x,drugname_l_y,prod_ai_l
0,100100247,10010024,1,PS,ADALIMUMAB,ADALIMUMAB,adalimumab,ADALIMUMAB,ADALIMUMAB
1,100100247,10010024,2,SS,ADALIMUMAB,ADALIMUMAB,adalimumab,ADALIMUMAB,ADALIMUMAB
2,100100247,10010024,3,SS,CYCLOSPORINE,CYCLOSPORINE,cyclosporine,CYCLOSPORINE,CYCLOSPORINE
3,100100247,10010024,3,SS,CYCLOSPORINE,CYCLOSPORINE,cyclosporine,CYCLOSPORINE,NaN
4,100100247,10010024,4,SS,REMICADE,INFLIXIMAB,remicade,REMICADE,INFLIXIMAB


In [48]:
df_raw_4.isna().sum()

primaryid            0
caseid               0
drug_seq             0
role_cod             0
drugname             0
prod_ai          30920
drugname_l_x         0
drugname_l_y         0
prod_ai_l       628715
dtype: int64

In [49]:
lookup_2[lookup_2['drugname_l'] == 'CYCLOSPORINE']

,drugname_l,prod_ai_l
2,CYCLOSPORINE,CYCLOSPORINE
1216178,CYCLOSPORINE,NaN
